Notebook criado pela @Mariana e @Heber

In [ ]:
# pop residente por sexo e grupos de idade
https://servicodados.ibge.gov.br/api/v3/agregados/205/periodos/1991/variaveis/93?localidades=N6[all]&classificacao=2[4,5]|58[all]

#pop residente por sexo, situaçao e grupos de idade-amostra-caracteristicas gerais da pop
https://servicodados.ibge.gov.br/api/v3/agregados/200/periodos/2010/variaveis/93?localidades=N6[all]&classificacao=2[4,5]|1[0]|58[all]

#Valor do rendimento nominal médio mensal domiciliar per capita das pessoas de 10 anos ou mais de idade, com rendimento mensal domiciliar per capita, residentes em domicílios particulares, exclusive as pessoas cuja condição no domicílio era pensionista, empregado(a) doméstico(a) ou parente do(a) empregado(a) doméstico(a)
https://servicodados.ibge.gov.br/api/v3/agregados/3974/periodos/2010/variaveis/3948?localidades=N6[all]&classificacao=12085[100543]|58[95253]


#puxar escolaridade
#puxar dados de renda


In [3]:
import urllib3
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [4]:
#HTTP client para python
http = urllib3.PoolManager()

lista_grupos_idades = [0, 1140, 2483]

#urlApiIBGE = 'http://servicodados.ibge.gov.br/api/v3/agregados/205/periodos/1991/variaveis/93?localidades=N6[all]&classificacao=2[4,5]|58[0]'
#response = http.request('GET', urlApiIBGE)
#print('responde status:', response.status)
lista_resultados = []
for grupo_idade in lista_grupos_idades:
  urlApiIBGE = f'http://servicodados.ibge.gov.br/api/v3/agregados/205/periodos/1991/variaveis/93?localidades=N6[all]&classificacao=2[4,5]|58[{grupo_idade}]'
  response = http.request('GET', urlApiIBGE)
  data_response = response.data.decode('utf-8')
  data_json = json.loads(data_response)
  print(f'responde status {grupo_idade}: {response.status}')
  for resultado in data_json[0]['resultados']:
    codigo_sexo = list(resultado['classificacoes'][0]['categoria'].keys())[0]
    valor_sexo = resultado['classificacoes'][0]['categoria'][codigo_sexo]
    codigo_categoria = list(resultado['classificacoes'][1]['categoria'].keys())[0]  
    valor_categoria = resultado['classificacoes'][1]['categoria'][codigo_categoria]      
    for serie in resultado['series']:
        codigo_local = serie['localidade']['id']
        nome_local = serie['localidade']['nome']
        ano = list(serie['serie'].keys())[0]
        valor = serie['serie'][ano]
        lista_resultados.append(
            {
              'codigo_sexo':codigo_sexo,
              'valor_sexo':valor_sexo,
              'codigo_local':codigo_local,           
              'nome_local':nome_local,
              'ano':ano,
              'populacao':valor,           
              'faixa_idade':valor_categoria
            }
        )

responde status 0: 200
responde status 1140: 200
responde status 2483: 200


In [5]:
exemplo_dados = pd.DataFrame.from_records(lista_resultados)
exemplo_dados

,codigo_sexo,valor_sexo,codigo_local,nome_local,ano,populacao,faixa_idade
0,4,Homens,1100015,Alta Floresta D'Oeste - RO,1991,17009,Total
1,4,Homens,1100023,Ariquemes - RO,1991,44267,Total
2,4,Homens,1100031,Cabixi - RO,1991,4348,Total
3,4,Homens,1100049,Cacoal - RO,1991,40612,Total
4,4,Homens,1100056,Cerejeiras - RO,1991,11230,Total
...,...,...,...,...,...,...,...
26941,5,Mulheres,5221809,Urutaí - GO,1991,27,0 ano
26942,5,Mulheres,5221908,Varjão - GO,1991,31,0 ano
26943,5,Mulheres,5222005,Vianópolis - GO,1991,83,0 ano
26944,5,Mulheres,5222054,Vicentinópolis - GO,1991,63,0 ano
